<a href="https://colab.research.google.com/github/Likelipop/03-IML/blob/main/Week_08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#cài đặt thư viện
!pip install -q lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.0/819.0 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 962.5/962.5 kB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
#download dataset
#!wget https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv
!gdown 1aXs9anuFLEOO2iQ9mUYDOkni04v_Fy9J

Downloading...
From: https://drive.google.com/uc?id=1aXs9anuFLEOO2iQ9mUYDOkni04v_Fy9J
To: /content/iris.csv
100% 3.98k/3.98k [00:00<00:00, 13.3MB/s]


In [3]:
import lightning as L
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

In [4]:
#read dataset
data = pd.read_csv("/content/iris.csv")
data.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


Iris là dataset nhận dạng loại hoa từ 4 features bao gồm sepal.length, sepal.widt, petal.length, và petal.width

In [5]:
#encode cho label
labels = {}
for index, element in enumerate(data["variety"].unique()):
  labels[element] = index

data.loc[:, "labels"] = data["variety"].apply(lambda x: labels[x])
data.head()

,sepal.length,sepal.width,petal.length,petal.width,variety,labels
0,5.1,3.5,1.4,0.2,Setosa,0
1,4.9,3.0,1.4,0.2,Setosa,0
2,4.7,3.2,1.3,0.2,Setosa,0
3,4.6,3.1,1.5,0.2,Setosa,0
4,5.0,3.6,1.4,0.2,Setosa,0


In [6]:
# Tạo class Dataset để đọc dữ liệu
COLUMNS = data.drop(['variety', "labels"], axis=1).columns.to_list()  # Lấy danh sách các cột không bao gồm 'variety' và 'labels'

class DataSet(torch.utils.data.Dataset):
    def __init__(self, data, normalizer, columns=COLUMNS):
        super(DataSet, self).__init__()  # Khởi tạo lớp cha
        self.data = data  # Lưu trữ dữ liệu đầu vào
        self.feature = normalizer.transform(self.data[columns].values)  # Chuẩn hóa các đặc trưng
        self.feature = torch.tensor(self.feature).float()  # Chuyển đổi các đặc trưng thành tensor kiểu float
        self.label = torch.tensor(self.data["labels"].values)  # Chuyển đổi nhãn thành tensor

    def __len__(self):
        return len(self.feature)  # Trả về số lượng mẫu trong dataset

    def __getitem__(self, idx):
        # Trả về một từ điển chứa đặc trưng và nhãn tại chỉ số idx
        return {"feature": self.feature[idx],
                "label": self.label[idx]}


In [7]:
# Chia và chuẩn hóa dataset
from sklearn.preprocessing import StandardScaler  # Nhập StandardScaler từ thư viện sklearn

BATCH_SIZE = 32  # Kích thước batch cho DataLoader
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra (80% - 20%)
train_data, test_data = train_test_split(data, test_size=0.2, random_state=0)
# Chia tập huấn luyện thành tập kiểm tra và tập val (60% train, 20% test 20% val)
test_data, val_data = train_test_split(train_data, test_size=0.25, random_state=0)

# Khởi tạo StandardScaler để chuẩn hóa dữ liệu
normalizer = StandardScaler()
# Tính toán các tham số chuẩn hóa dựa trên tập huấn luyện
normalizer.fit(train_data[COLUMNS].values)

StandardScaler()

In [8]:
# Tạo DataLoader cho tập huấn luyện, cho phép trộn và sử dụng nhiều workers
train_loader = torch.utils.data.DataLoader(DataSet(train_data, normalizer), batch_size=BATCH_SIZE,
                                           shuffle=True, num_workers=2)

# Tạo DataLoader cho tập val
val_loader = torch.utils.data.DataLoader(DataSet(val_data, normalizer), batch_size=BATCH_SIZE, num_workers=2)

# Tạo DataLoader cho tập kiểm tra
test_loader = torch.utils.data.DataLoader(DataSet(test_data, normalizer), batch_size=BATCH_SIZE, num_workers=2)

# Lấy một batch dữ liệu từ train_loader
data_loader = next(iter(train_loader))
# In ra kích thước của các đặc trưng trong batch
print(len(data_loader["feature"]))
# In ra kích thước của các nhãn trong batch
print(len(data_loader["label"]))
# In ra các đặc trưng và nhãn (bình luận lại để không in ra)
#print(data_loader["feature"], "\n", data_loader["label"])


32
32


In [9]:
len(COLUMNS)

4

In [10]:
# Tạo model
class Model(L.LightningModule):
    def __init__(self, num_classes=len(labels), learning_rate=5e-3, input_dim=len(COLUMNS)):
        super(Model, self).__init__()  # Khởi tạo lớp cha
        self.learning_rate = learning_rate  # Lưu trữ learning rate
        # Tạo mô hình MLP với các lớp tuyến tính và hàm kích hoạt ReLU
        self.mlp = torch.nn.Sequential(
            torch.nn.Linear(input_dim, 64),  # Lớp đầu vào với kích thước input_dim và đầu ra 64
            torch.nn.ReLU(),                  # Hàm kích hoạt ReLU
            torch.nn.Linear(64, 32),          # Lớp tiếp theo với đầu vào 64 và đầu ra 32
            torch.nn.ReLU(),                  # Hàm kích hoạt ReLU
            torch.nn.Linear(32, num_classes), # Lớp đầu ra với đầu vào 32 và đầu ra số lớp
        )

    def forward(self, x):
        return self.mlp(x)  # Thực hiện phép biến đổi thông qua mô hình MLP

    def training_step(self, batch, batch_idx):
        x = batch["feature"]  # Lấy đặc trưng từ batch
        y = batch["label"]    # Lấy nhãn từ batch
        y_pred = self(x)      # Dự đoán nhãn bằng mô hình
        # Tính toán loss bằng hàm CrossEntropy
        loss = torch.nn.CrossEntropyLoss()(y_pred, y)
        self.log('train_loss', loss, prog_bar=True)  # Ghi lại loss huấn luyện để hiển thị
        return loss  # Trả về loss

    def validation_step(self, batch, batch_idx):
        x = batch["feature"]  # Lấy đặc trưng từ batch
        y = batch["label"]    # Lấy nhãn từ batch
        y_pred = self(x)      # Dự đoán nhãn bằng mô hình
        # Tính toán loss cho tập val
        val_loss = torch.nn.CrossEntropyLoss()(y_pred, y)
        self.log('val_loss', val_loss, prog_bar=True)  # Ghi lại loss val để hiển thị
        return val_loss  # Trả về loss val

    def configure_optimizers(self):
        # Khởi tạo optimizer Adam với learning rate đã định
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer  # Trả về optimizer


In [11]:
# Train model
model = Model()  # Khởi tạo một thể hiện của mô hình

# Tạo Trainer từ PyTorch Lightning với số epoch tối đa là 100 và kích hoạt tính năng phát hiện bất thường
trainer = L.Trainer(max_epochs=100, detect_anomaly=True)

# Bắt đầu quá trình huấn luyện mô hình với dữ liệu huấn luyện và val
trainer.fit(model, train_loader, val_loader)


INFO: Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
INFO:lightning.pytorch.utilities.rank_zero:Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
INFO: You have turned on `Trainer(detect_anomaly=True)`. This will significantly slow down compute speed and is recommended only for model debugging.
INFO:lightning.pytorch.utilities.rank_zero:You have turned on `Trainer(detect_anomaly=True)`. This will significantly slow down compute speed and is recommended only for model debugging.
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPU

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (4) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=100` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.


#Bài tập

Hãy thêm def test_step vào class Model, tiến hành huấn luyện model với batch size là 64, lr là 5e-4, epochs là 50.



In [62]:
# Tạo model
class Model(L.LightningModule):
    def __init__(self, num_classes=len(labels), learning_rate=5e-3, input_dim=len(COLUMNS)):
        super(Model, self).__init__()  # Khởi tạo lớp cha
        self.learning_rate = learning_rate  # Lưu trữ learning rate
        # Tạo mô hình MLP với các lớp tuyến tính và hàm kích hoạt ReLU
        self.mlp = torch.nn.Sequential(
            torch.nn.Linear(input_dim, 64),  # Lớp đầu vào với kích thước input_dim và đầu ra 64
            torch.nn.ReLU(),                  # Hàm kích hoạt ReLU
            torch.nn.Linear(64, 32),          # Lớp tiếp theo với đầu vào 64 và đầu ra 32
            torch.nn.ReLU(),                  # Hàm kích hoạt ReLU
            torch.nn.Linear(32, num_classes), # Lớp đầu ra với đầu vào 32 và đầu ra số lớp
        )

    def forward(self, x):
        return self.mlp(x)  # Thực hiện phép biến đổi thông qua mô hình MLP

    def training_step(self, batch, batch_idx):
        x = batch["feature"]  # Lấy đặc trưng từ batch
        y = batch["label"]    # Lấy nhãn từ batch
        y_pred = self(x)      # Dự đoán nhãn bằng mô hình
        # Tính toán loss bằng hàm CrossEntropy
        loss = torch.nn.CrossEntropyLoss()(y_pred, y)
        self.log(f'train_loss {batch_idx}', loss, prog_bar=True)  # Ghi lại loss huấn luyện để hiển thị
        return loss  # Trả về loss

    def validation_step(self, batch, batch_idx):
        x = batch["feature"]  # Lấy đặc trưng từ batch
        y = batch["label"]    # Lấy nhãn từ batch
        y_pred = self(x)      # Dự đoán nhãn bằng mô hình
        # Tính toán loss cho tập val
        val_loss = torch.nn.CrossEntropyLoss()(y_pred, y)
        self.log(f'val_loss {batch_idx}', val_loss, prog_bar=True)  # Ghi lại loss val để hiển thị
        return val_loss  # Trả về loss val

    def configure_optimizers(self):
        # Khởi tạo optimizer Adam với learning rate đã định
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer  # Trả về optimizer

    def test_step(self, batch, batch_idx):
      X = batch["feature"]
      y = batch["label"]
      y_pred = self(X)
      test_loss = torch.nn.CrossEntropyLoss()(y_pred, y)
      self.log(f'test_loss {batch_idx}', test_loss, prog_bar = True)
      return test_loss


In [26]:
BATCH_SIZE = 64
lr = 5e-4
NUM_EPOCHS = 50

In [63]:
# Tạo DataLoader cho tập huấn luyện, cho phép trộn và sử dụng nhiều workers
train_loader = torch.utils.data.DataLoader(DataSet(train_data, normalizer), batch_size=BATCH_SIZE,
                                           shuffle=True, num_workers=2)

# Tạo DataLoader cho tập val
val_loader = torch.utils.data.DataLoader(DataSet(val_data, normalizer), batch_size=BATCH_SIZE, num_workers=2)

# Tạo DataLoader cho tập kiểm tra
test_loader = torch.utils.data.DataLoader(DataSet(test_data, normalizer), batch_size=BATCH_SIZE, num_workers=2)

# Lấy một batch dữ liệu từ train_loader
data_loader = next(iter(train_loader))
# In ra kích thước của các đặc trưng trong batch
print(len(data_loader["feature"]))
# In ra kích thước của các nhãn trong batch
print(len(data_loader["label"]))
# In ra các đặc trưng và nhãn (bình luận lại để không in ra)
#print(data_loader["feature"], "\n", data_loader["label"])


64
64


In [64]:
# Train model
model = Model(learning_rate  = lr)  # Khởi tạo một thể hiện của mô hình

# Tạo Trainer từ PyTorch Lightning với số epoch tối đa là 100 và kích hoạt tính năng phát hiện bất thường
trainer = L.Trainer(max_epochs=NUM_EPOCHS, detect_anomaly=True)

# Bắt đầu quá trình huấn luyện mô hình với dữ liệu huấn luyện và val
trainer.fit(model, train_loader, val_loader)


INFO: Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
INFO:lightning.pytorch.utilities.rank_zero:Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
INFO: You have turned on `Trainer(detect_anomaly=True)`. This will significantly slow down compute speed and is recommended only for model debugging.
INFO:lightning.pytorch.utilities.rank_zero:You have turned on `Trainer(detect_anomaly=True)`. This will significantly slow down compute speed and is recommended only for model debugging.
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPU

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (2) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=50` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=50` reached.


In [44]:
trainer.test(model, test_loader)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.4714149832725525     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.4714149832725525}]

In [85]:
from sklearn.metrics import classification_report
import torch
from collections import Counter
model.eval()  # Set the model to evaluation mode
all_labels = []
all_predictions = []

with torch.no_grad():  # Disable gradient calculations
    for batch in test_loader:
        inputs = batch["feature"]
        labels_tensor = batch["label"] # Rename the tensor variable to avoid conflict
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)

        all_labels.extend(labels_tensor.cpu().numpy()) # Use the tensor here
        all_predictions.extend(predicted.cpu().numpy())

# Generate the classification report
# Use the original labels dictionary to get the class names
# The original labels dictionary is named 'labels' in your environment
my_counter = Counter(labels.tolist())
target_names = list(str(i) for i in my_counter.keys()) # This 'labels' should refer to the dictionary
report = classification_report(all_labels, all_predictions, target_names=target_names)
print(report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        29
           2       0.95      0.69      0.80        29
           1       0.78      0.97      0.86        32

    accuracy                           0.89        90
   macro avg       0.91      0.89      0.89        90
weighted avg       0.90      0.89      0.89        90

